In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
import logging

# 로그 설정  
logging.basicConfig(level=logging.DEBUG)

# CSV 파일 경로
csv_file_path = 'munpia_novel_data.csv'

def load_data():
    try:
        # CSV 파일에서 데이터를 로드합니다.
        df = pd.read_csv(csv_file_path)
        
        # 소설의 제목, 저자, 소개, 태그를 결합하여 새로운 피처를 만듭니다.
        df['combined_features'] = df['title'] + ' ' + df['author'] + ' ' + df['intro'] + ' ' + df['tags']
        df['combined_features'] = df['combined_features'].fillna('')  # 결측값을 공백으로 대체

        # TF-IDF 벡터화
        vectorizer = TfidfVectorizer()
        combined_matrix = vectorizer.fit_transform(df['combined_features'])
        return df, combined_matrix
    except Exception as e:
        logging.error("Error loading or processing data from CSV file: %s", e)
        return None, None

# TF-IDF로 데이터를 로드
df, combined_matrix = load_data()
if df is None or combined_matrix is None:
    logging.error("Failed to load data. Exiting.")
    exit()

def recommend_novel(title, combined_matrix, df):
    try:
        idx = df.index[df['title'] == title].tolist()
        if not idx:
            logging.debug("Title not found: %s", title)
            return []
        idx = idx[0]

        # KNN 모델을 사용하여 추천
        knn = NearestNeighbors(metric='cosine', algorithm='brute')
        knn.fit(combined_matrix)
        distances, indices = knn.kneighbors(combined_matrix[idx], n_neighbors=11)
        sim_scores = list(zip(indices.flatten(), 1 - distances.flatten()))
        sim_scores = sim_scores[1:]  # 자신을 제외한 상위 10개 추천

        novel_indices = [i[0] for i in sim_scores]
        recommendations = df.loc[novel_indices, ['title', 'author', 'link']]
        recommendations['rank'] = range(1, len(recommendations) + 1)
        return recommendations.to_dict(orient='records')
    except Exception as e:
        logging.error("Error in recommend_novel function: %s", e)
        return []

def evaluate_model(test_titles, combined_matrix, df):
    correct_predictions = 0
    total_predictions = len(test_titles)

    for title in test_titles:
        recommendations = recommend_novel(title, combined_matrix, df)
        recommended_titles = [rec['title'] for rec in recommendations]
        if title in recommended_titles:
            correct_predictions += 1

    accuracy = correct_predictions / total_predictions
    logging.info("Model Accuracy: %.2f%%", accuracy * 100)
    return accuracy

def main():
    # 데이터셋을 훈련 세트와 테스트 세트로 분할
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    test_titles = test_df['title'].tolist()

    # 정확도 평가
    logging.info("Evaluating the KNN model...")
    accuracy = evaluate_model(test_titles, combined_matrix, df)
    logging.info("KNN Model Accuracy: %.2f%%", accuracy * 100)

    # 사용자 입력으로 추천을 요청
    title = input("Enter the title for recommendations: ")
    recommendations = recommend_novel(title, combined_matrix, df)
    if recommendations:
        print("\nRecommendations:")
        for rec in recommendations:
            print(f"Rank {rec['rank']}: {rec['title']} by {rec['author']}")
            print(f"Link: {rec['link']}\n")
    else:
        print("No recommendations found.")

if __name__ == "__main__":
    main()


INFO:root:Evaluating the KNN model...
INFO:root:Model Accuracy: 43.67%
INFO:root:KNN Model Accuracy: 43.67%



Recommendations:
Rank 1: 선독점 천조국은 이제 제겁니다 by 코알라
Link: https://novel.munpia.com/409689

Rank 2: 선독점 불곰국은 이제 제겁니다 by 코알라
Link: https://novel.munpia.com/362833

Rank 3: 선독점 불곰국은 이제 제겁니다 by 코알라
Link: https://novel.munpia.com/362833

Rank 4: 선독점 불곰국은 이제 제겁니다 by 코알라
Link: https://novel.munpia.com/362833

Rank 5: 선독점 불곰국은 이제 제겁니다 by 코알라
Link: https://novel.munpia.com/362833

Rank 6: 선독점 역대급 천재 재벌 by 코알라
Link: https://novel.munpia.com/171553

Rank 7: 선독점 역대급 천재 재벌 by 코알라
Link: https://novel.munpia.com/171553

Rank 8: 선독점 나 혼자 천재 재벌로 레벨업(케미 천마) by 코알라
Link: https://novel.munpia.com/253899

Rank 9: 선독점 나 혼자 천재 재벌로 레벨업(케미 천마) by 코알라
Link: https://novel.munpia.com/253899

Rank 10: 선독점 천조국 건스미스가 되었다 by [서평]
Link: https://novel.munpia.com/412508



In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
import logging

# 로그 설정
logging.basicConfig(level=logging.DEBUG)

# CSV 파일 경로
csv_file_path = 'munpia_novel_data.csv'

def load_data():
    try:
        # CSV 파일에서 데이터를 로드합니다.
        df = pd.read_csv(csv_file_path)
        
        # 결측값을 처리합니다.
        df['title'] = df['title'].fillna('')
        df['author'] = df['author'].fillna('')
        df['intro'] = df['intro'].fillna('')
        df['tags'] = df['tags'].fillna('')
        
        # 제목을 기준으로 중복된 작품 제거 (가장 처음 등장한 행만 남김)
        df = df.drop_duplicates(subset=['title'], keep='first')
        
        # 소설의 제목, 저자, 소개, 태그를 결합하여 새로운 피처 생성
        df['combined_features'] = df['title'] + ' ' + df['author'] + ' ' + df['intro'] + ' ' + df['tags']
        return df
    except Exception as e:
        logging.error("Error loading or processing data from CSV file: %s", e)
        return None

# 데이터를 로드합니다.
df = load_data()
if df is None:
    logging.error("Failed to load data. Exiting.")
    exit()

# TF-IDF Vectorizer를 사용하여 결합된 특성 벡터화
tfidf_vectorizer = TfidfVectorizer(max_df=0.85, min_df=2, stop_words='english', ngram_range=(1, 2))
combined_matrix = tfidf_vectorizer.fit_transform(df['combined_features'])

# KNN 모델 학습
knn_model = NearestNeighbors(n_neighbors=11, metric='cosine')
knn_model.fit(combined_matrix)

# 소설 추천 함수 (KNN 모델 사용)
def recommend_novel(title):
    try:
        if title not in df['title'].values:
            logging.debug("Title not found: %s", title)
            return [], []
        
        idx = df.index[df['title'] == title].tolist()[0]
        
        # 입력한 소설과 유사한 소설 찾기 (자기 자신 포함)
        distances, indices = knn_model.kneighbors(combined_matrix[idx].reshape(1, -1))
        
        # 자기 자신을 제외한 상위 10개 추천
        novel_indices = indices[0][1:11]
        
        # 중복된 소설을 제거합니다.
        novel_indices = list(dict.fromkeys(novel_indices))
        
        # 추천된 소설과 입력 소설 간의 코사인 유사도 계산
        similarity_scores = cosine_similarity(combined_matrix[idx], combined_matrix[novel_indices])[0]
        
        # 유사도 0.05 이상만 필터링 (유사도가 너무 낮은 소설은 제외)
        filtered_indices = [i for i, score in zip(novel_indices, similarity_scores) if score > 0.05]
        filtered_scores = [score for score in similarity_scores if score > 0.05]
        
        # 추천된 소설 출력
        recommendations = df[['title', 'link']].iloc[filtered_indices]
        return recommendations.to_dict(orient='records'), filtered_scores
    except Exception as e:
        logging.error("Error in recommend_novel function: %s", e)
        return [], []

# 사용자 입력을 받아서 추천 소설 출력
def get_novel_recommendations():
    title = input("추천받고 싶은 소설 제목을 입력하세요: ")
    recommendations, similarity_scores = recommend_novel(title)
    
    if not recommendations:
        print("해당 제목의 소설을 찾을 수 없습니다.")
    else:
        print(f"'{title}'와(과) 유사한 소설 Top 10 추천:")
        for idx, (rec, score) in enumerate(zip(recommendations, similarity_scores), 1):
            print(f"{idx}. {rec['title']} (유사도: {score:.2f})")

# 프로그램 실행
if __name__ == "__main__":
    get_novel_recommendations()


'선독점 전지적 독자 시점'와(과) 유사한 소설 Top 10 추천:
1. 선독점 전지적 관중 시점 (유사도: 0.33)
2. 선독점 나 혼자 게이트 속 세상을 알고 있다. (유사도: 0.33)
3. 선독점 코딩의 신 IT재벌 되다 (유사도: 0.29)
4. 선독점 킬 더 히어로 (유사도: 0.28)
5. 선독점 실전된 무공으로 먼치킨 (유사도: 0.24)
6. 선독점 대마도사 서자는 전생 플레이어 (유사도: 0.22)
7. 선독점 시스템 에러로 종족초월 (유사도: 0.22)
8. 선독점 멸망한 세계의 커뮤니티 사용법 (유사도: 0.21)
9. 선독점 스킬조합으로 꿀빠는 공무헌터 (유사도: 0.18)
10. 선독점 신이 내린 방송천재 (유사도: 0.17)
